In [105]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, RobustScaler
from xgboost import XGBRegressor
import seaborn as sns
import matplotlib.pyplot as plt

In [106]:
Fact_Sales = pd.read_csv(r"D:\Data Analysis BC\Projects\Project4\Data-Analysis-Projects-3-SuperStore\Fact&dim-csv\Fact_Sales.csv")
Ship_Mode = pd.read_csv(r"D:\Data Analysis BC\Projects\Project4\Data-Analysis-Projects-3-SuperStore\Fact&dim-csv\Dim_Ship_Mode.csv")
Product = pd.read_csv(r"D:\Data Analysis BC\Projects\Project4\Data-Analysis-Projects-3-SuperStore\Fact&dim-csv\Dim_Product.csv")
Priority = pd.read_csv(r"D:\Data Analysis BC\Projects\Project4\Data-Analysis-Projects-3-SuperStore\Fact&dim-csv\Dim_Priority.csv")
Market = pd.read_csv(r"D:\Data Analysis BC\Projects\Project4\Data-Analysis-Projects-3-SuperStore\Fact&dim-csv\Dim_Market.csv")
Geography = pd.read_csv(r"D:\Data Analysis BC\Projects\Project4\Data-Analysis-Projects-3-SuperStore\Fact&dim-csv\Dim_Geography.csv")
Date = pd.read_csv(r"D:\Data Analysis BC\Projects\Project4\Data-Analysis-Projects-3-SuperStore\Fact&dim-csv\Dim_Date.csv")
Customer = pd.read_csv(r"D:\Data Analysis BC\Projects\Project4\Data-Analysis-Projects-3-SuperStore\Fact&dim-csv\Dim_Customer.csv")

In [107]:
merged = pd.merge(Fact_Sales , Priority , how='inner' , on='Priority Key')
merged = pd.merge(merged , Product , how='inner' , on='Product Key')
merged = pd.merge(merged , Ship_Mode , how='inner' , on='Ship Mode Key')
merged = pd.merge(merged , Market , how='inner' , on='Market Key')
merged = pd.merge(merged , Geography , how='inner' , on='Geo Key')
merged = pd.merge(merged, Date, left_on='Ship Date Key', right_on='Date Key', how='inner')
merged = pd.merge(merged , Customer , how='inner' , on='Customer Key')
merged = merged[merged['Is Returned'] == 0]

## Feature engineering

In [108]:
extract_df = merged.iloc[: , [1, 2, 3, 4, 5 ,15 , 18 , 20 , 21, 24 , 29, 37]]

In [109]:
#pre processing

#extract_df['Quarter'] = extract_df['Quarter'].map({1 : 'spring' , 2 : 'summer' , 3 : 'fall' , 4 : 'winter'})

top_10_countries = extract_df['Country'].value_counts().nlargest(10).index
extract_df['Country_Grouped'] = extract_df['Country'].where(extract_df['Country'].isin(top_10_countries), 'Other')

C:\Users\Asus\AppData\Local\Temp\ipykernel_532\4124163530.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract_df['Country_Grouped'] = extract_df['Country'].where(extract_df['Country'].isin(top_10_countries), 'Other')


In [110]:
dummied_df = extract_df.copy()

df_priority_dummy = pd.get_dummies(
    data=extract_df['Priority Name'],
    drop_first=True,
    prefix='priority',
    prefix_sep='_',
    dtype=int
)
df_subcat_dummy = pd.get_dummies(
    data=extract_df['Sub Category'],
    drop_first=True,
    prefix='sub_cat',
    prefix_sep='_',
    dtype=int
)
df_shipmode_name_dummy = pd.get_dummies(
    data=extract_df['Ship Mode Name'],
    drop_first=True,
    prefix='SMN',
    prefix_sep='_',
    dtype=int
)
df_market_dummy = pd.get_dummies(
    data=extract_df['Market'],
    drop_first=True,
    prefix='market',
    prefix_sep='_',
    dtype=int
)
df_segment_dummy = pd.get_dummies(
    data=extract_df['Segment'],
    drop_first=True,
    prefix='segment',
    prefix_sep='_',
    dtype=int
)
'''df_region_dummy = pd.get_dummies(
    data=extract_df['Region'],
    drop_first=True,
    prefix='region',
    prefix_sep='_',
    dtype=int
)'''
'''df_quarter_dummy = pd.get_dummies(
    data=extract_df['Quarter'],
    drop_first=True,
    prefix='quarter',
    prefix_sep='_',
    dtype=int
)'''
df_country_dummy = pd.get_dummies(
    data=extract_df['Country_Grouped'],
    drop_first=True,
    prefix='country',
    prefix_sep='_',
    dtype=int
)
dummied_df = dummied_df.join(df_priority_dummy)
dummied_df = dummied_df.join(df_subcat_dummy)
dummied_df = dummied_df.join(df_shipmode_name_dummy)
dummied_df = dummied_df.join(df_market_dummy)
dummied_df = dummied_df.join(df_segment_dummy)
'''dummied_df = dummied_df.join(df_region_dummy)'''
'''dummied_df = dummied_df.join(df_quarter_dummy)'''
dummied_df = dummied_df.join(df_country_dummy)
dummied_df = dummied_df.drop(columns=['Segment' , 'Market' , 'Ship Mode Name' , 'Sub Category' , 'Priority Name' ,'Country', 'Country_Grouped'])

In [111]:
# inventing new features (commented features confused the model)
featured_df = dummied_df.copy()

'''featured_df['Price_per_Unit'] = featured_df['Sales'] / featured_df['Quantity']'''


'''epsilon = 1e-9
featured_df['Original_Sales'] = featured_df['Sales'] / (1 - featured_df['Discount'] + epsilon)
featured_df['Discount_Amount'] = featured_df['Original_Sales'] * featured_df['Discount']'''


start_year = featured_df['Year'].min()
featured_df['Years_Since_Start'] = featured_df['Year'] - start_year
featured_df.drop(columns=['Year'], inplace=True)


# creating a binary flag for high value sales and high quantities because they follow different patterns as we saw in hypothesis tests =>
sales_threshold = featured_df['Sales'].quantile(0.6)
featured_df['Is_High_Value_Order'] = (featured_df['Sales'] > sales_threshold).astype(int)

'''quantity_threshold = featured_df['Quantity'].quantile(0.95)
featured_df['Is_High_Quantity_Order'] = (featured_df['Quantity'] > quantity_threshold).astype(int)'''

discount_threshold = featured_df['Discount'].quantile(0.65)
featured_df['Is_high_discount'] = (featured_df['Discount'] > discount_threshold).astype(int)


'''freq_map = featured_df['Country'].value_counts().to_dict()
featured_df['Country_Freq_Encoded'] = featured_df['Country'].map(freq_map)
featured_df.drop('Country', axis=1 , inplace=True)'''


featured_df['powered_discount'] = featured_df['Discount'] ** 2


# sum sales of each market
market_cols = [col for col in featured_df.columns if col.startswith('market_')]
original_market_col = featured_df[market_cols].idxmax(axis=1)
featured_df['market_original'] = original_market_col.str.replace('market_', '')
market_sales_map = featured_df.groupby('market_original')['Sales'].sum().to_dict()
featured_df['sum_sale_of_each_market'] = featured_df['market_original'].map(market_sales_map)
featured_df = featured_df.drop('market_original', axis=1)

In [112]:
corr_matrix = featured_df.corr().abs() * 100 
columns = corr_matrix.columns
high_correlated_pairs = []

for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        
        feature1 = columns[i]
        feature2 = columns[j]
            
        correlation_value = corr_matrix.loc[feature1, feature2]
        corr_with_target1 = corr_matrix.loc[feature1 , 'Profit']
        corr_with_target2 = corr_matrix.loc[feature2 , 'Profit']
        
        if correlation_value > 50:
            print(f"{feature1} & {feature2} : {correlation_value:.1f}")
            print(f'{feature1} corr with target : {corr_with_target1 : .1f}')
            print(f'{feature2} corr with target : {corr_with_target2 : .1f}')
            print('')

Sales & Shipping Cost : 77.2
Sales corr with target :  47.6
Shipping Cost corr with target :  35.9

Discount & Is_high_discount : 85.2
Discount corr with target :  32.0
Is_high_discount corr with target :  27.4

Discount & powered_discount : 95.1
Discount corr with target :  32.0
powered_discount corr with target :  30.6

priority_High & priority_Medium : 76.7
priority_High corr with target :  0.5
priority_Medium corr with target :  0.3

SMN_Second Class & SMN_Standard Class : 61.6
SMN_Second Class corr with target :  0.2
SMN_Standard Class corr with target :  0.2

market_EMEA & sum_sale_of_each_market : 55.5
market_EMEA corr with target :  3.4
sum_sale_of_each_market corr with target :  2.9

market_US & country_United States : 100.0
market_US corr with target :  0.3
country_United States corr with target :  0.3

Is_high_discount & powered_discount : 68.8
Is_high_discount corr with target :  27.4
powered_discount corr with target :  30.6



In [113]:
featured_df = featured_df.drop(columns=['market_US' , 'SMN_Second Class' , 'Discount' ,'market_EMEA'])

In [114]:
'''IQR_df = featured_df.copy()
for col in ['Profit']:
    Q1 = IQR_df[col].quantile(0.25 , interpolation='midpoint')
    Q3 = IQR_df[col].quantile(0.75 , interpolation='midpoint')
    IQR = Q3 - Q1
    up_lim = Q3 + (1.5 * IQR)
    low_lim = Q1 - (1.5 * IQR)
    IQR_df = IQR_df.loc[
    (IQR_df[col] >= low_lim) &
    (IQR_df[col] <= up_lim)
]'''

final_data = featured_df.copy()

## XGBoost Regression

In [115]:
data_for_XGB_reg = final_data.copy()
X_base = data_for_XGB_reg[[col for col in data_for_XGB_reg.columns if col != 'Profit' and col != 'Shipping Cost']]
Y_base = data_for_XGB_reg.iloc[:, 3]
X_train, X_test, Y_train, Y_test = train_test_split(
    X_base,
    Y_base,
    train_size=0.75,
    #random_state=444
)

robust = RobustScaler()
X_train_rbs = robust.fit_transform(X_train)
X_test_rbs = robust.transform(X_test)

model_xgboost = XGBRegressor(
    booster='gbtree',
    tree_method = "hist",
    device = "cuda",
    objective='reg:squarederror',
    #random_state=444,
    early_stopping_rounds=50,
    n_estimators=1000,
    gamma=0,
    colsample_bytree=0.8,
    learning_rate=0.05,
    max_depth=4,
    min_child_weight=2,
    reg_alpha=0.1,
    reg_lambda=0.5,
    subsample=0.8
)

#   colsample_bytree=0.8,
#   learning_rate=0.05,
#   max_depth=3, + 
#   min_child_weight=1, + 
#   reg_alpha=0.05, +
#   reg_lambda=0.5,
#   subsample=0.7 +

'''{'colsample_bytree': 0.8,
 'learning_rate': 0.05,
 'max_depth': 4,
 'min_child_weight': 2,
 'reg_alpha': 0.1,
 'reg_lambda': 0.5,
 'subsample': 0.8}'''


'''param_grid = {
    'learning_rate': [0.03, 0.05],
    'max_depth': [3, 4],
    'min_child_weight': [1, 2],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8],
    'reg_alpha': [0.05, 0.1],
    'reg_lambda': [0.5, 0.65]
}

grid_cv = GridSearchCV(
    estimator=model_xgboost,
    param_grid=param_grid,
    scoring='r2',
    cv=3,
    n_jobs=1,
    verbose=2 
)
grid_cv.fit(X_train_rbs, Y_train, eval_set=[(X_test_rbs, Y_test)], verbose=False)'''


model_xgboost.fit(X_train_rbs, Y_train, eval_set=[(X_test_rbs, Y_test)], verbose=False)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [116]:
feature_importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model_xgboost.feature_importances_ * 100
})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)
feature_importance_df

,feature,importance
0,Sales,23.565392
40,Is_High_Value_Order,17.230537
4,priority_Medium,7.532242
22,SMN_Standard Class,6.137865
2,priority_High,3.312900
36,country_Other,3.002464
8,sub_cat_Bookcases,2.493162
17,sub_cat_Phones,2.274292
38,country_United States,1.897754
7,sub_cat_Binders,1.841737


In [117]:
y_pred_train = model_xgboost.predict(X_train_rbs)
r2_train = r2_score(Y_train, y_pred_train)
print(f"R2 score on Training set: {r2_train * 100 :.2f} %")

y_pred_test = model_xgboost.predict(X_test_rbs)
r2_test = r2_score(Y_test, y_pred_test)
print(f"R2 score on Test set: {r2_test * 100 :.2f} %")
print(f'overfit prob : {(r2_train * 100) - (r2_test * 100) : .2f} % (lower the better)')

R2 score on Training set: 82.96 %
R2 score on Test set: 82.24 %
overfit prob :  0.72 % (lower the better)
